In [79]:
import os
import requests
import json
import pandas as pd
from tqdm import tqdm

In [80]:
TTBKEY = 'ttbqkrdlstn97011854001'

In [81]:
CategoryId = [
    351, 2630, 2643, 2632, 2672, 2669, 7439, 2633, 2673, 2648, 2655, 2645, 55977, 55981, 55978, 
    55979, 55055, 55057, 55056, 2599, 2607, 2602, 2601, 2606, 2608, 2600, 6348, 2663, 2665, 2662, 
    7395, 2661, 6163, 6165, 2666, 6355, 128622, 128624, 128623, 2719, 2737, 6794, 2744, 2724, 2732, 
    2735, 3239, 2721, 5056, 5058, 5059, 5061, 5063, 6576, 6577, 6578, 7396, 6589, 2756, 7411, 427, 
    6593, 2065, 2509, 2510, 2684, 1714, 2753, 6188, 35322, 6627, 2687, 6720, 437, 6356, 6358, 6360, 
    6361, 6362, 6365, 6989, 2502, 6734, 6357, 14605, 6350, 6385, 6386, 7397, 35323, 2704, 6351, 
    15778, 6353, 6352, 6437, 479, 6892, 2836, 6995, 6996, 6997, 12984, 485, 2999, 2847, 7013, 7012, 
    7014, 6891, 483, 486, 7006, 7007, 5581, 68185, 36134, 6636, 2835, 6436, 2839, 2838, 2349, 2413, 
    2757, 2350, 363, 6369, 387, 6381, 2767, 2765, 49733, 2770, 2773, 2766, 2613, 6346, 6889, 6890, 
    2615, 3023, 2621
]

In [82]:
def make_book(cid, start, max_results,ttbkey=TTBKEY, QueryType="Bestseller"):
    url = "http://www.aladin.co.kr/ttb/api/ItemList.aspx"
    params = {
        "TTBKey": ttbkey,
        "QueryType": QueryType,
        "Version": "20131101",
        "SearchTarget": "Book",
        "CategoryId": cid,
        "start": start,
        "MaxResults": max_results,
        "output": "js",
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        # print(f"응답 데이터: {response.text}")
        # print(f"HTTP 응답 상태 코드: {response.status_code}")  # 상태 코드 확인
        # response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
        data = response.json()  # JSON 데이터 파싱
    except (requests.RequestException, json.JSONDecodeError) as e:
        print(f"API 호출 오류: {e}")
        return pd.DataFrame()  # 빈 데이터프레임 반환
    
    if "item" in data:
        items = [
            {
                "title": item.get("title", ""),
                "link": item.get("link", ""),
                "author": item.get("author", ""),
                "pubDate": item.get("pubDate", ""),
                "description": item.get("description", ""),
                "isbn": item.get("isbn", ""),
                "isbn13": item.get("isbn13", ""),
                "itemId": item.get("itemId", ""),
                "priceSales": item.get("priceSales", ""),
                "priceStandard": item.get("priceStandard", ""),
                "categoryId": item.get("categoryId", ""),
                "categoryName": item.get("categoryName", ""),
                "publisher": item.get("publisher", ""),
                "customerReviewRank": item.get("customerReviewRank", ""),
            }
            for item in data["item"]
        ]
        return pd.DataFrame(items)
    else:
        print("API 호출 성공, 그러나 반환된 데이터가 없습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

In [83]:
def collect_books_no_limit(CategoryId, ttbkey=TTBKEY):
    df = pd.DataFrame(columns=[
        "title", "link", "author", "pubDate", "description", "isbn", "isbn13",
        "itemId", "priceSales", "priceStandard", "categoryId", "categoryName",
        "publisher", "customerReviewRank",
    ])

    start = 1
    max_results_per_call = 50
    
    for cid in tqdm(CategoryId):
        new_data = make_book(cid, start=start, max_results=max_results_per_call)
        
        if new_data.empty:
            print(f"카테고리 ID : {cid} . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)")
        else:
            df = pd.concat([df, new_data]).drop_duplicates(subset=["itemId"], ignore_index=True)
    
    return df

In [84]:
book_db = collect_books_no_limit(CategoryId=CategoryId)

  4%|▍         | 6/143 [00:01<00:28,  4.77it/s]

카테고리 ID : 2672 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)


  6%|▌         | 8/143 [00:01<00:22,  5.95it/s]

카테고리 ID : 7439 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)


  8%|▊         | 12/143 [00:02<00:16,  8.11it/s]

...

카테고리 ID : 49733 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)
카테고리 ID : 2770 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)


 98%|█████████▊| 140/143 [00:21<00:00,  7.79it/s]

카테고리 ID : 6890 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)


100%|██████████| 143/143 [00:22<00:00,  6.42it/s]

카테고리 ID : 2621 . 데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)


In [85]:
len(book_db)

1193

In [8]:
def search_books(ttbkey, query="", queryType="ItemNewAll", start=1, max_results=50):
    url = "http://www.aladin.co.kr/ttb/api/ItemSearch.aspx"
    params = {
        "TTBKey": ttbkey,
        "Query": query,
        "QueryType": queryType,
        "SearchTarget": "Book",
        "CategoryId": "351",  # IT/컴퓨터 카테고리
        "start": start,
        "MaxResults": max_results,
        "Cover": "None",
        "output": "js",
        "Sort": "PublishTime",
        "Version": "20131101",
        "outofStockfilter": 1,
        "RecentPublishFilter": 1,
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        # print(f"응답 데이터: {response.text}")
        # print(f"HTTP 응답 상태 코드: {response.status_code}")  # 상태 코드 확인
        # response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
        data = response.json()  # JSON 데이터 파싱
    except (requests.RequestException, json.JSONDecodeError) as e:
        print(f"API 호출 오류: {e}")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    if "item" in data:
        items = [
            {
                "title": item.get("title", ""),
                "link": item.get("link", ""),
                "author": item.get("author", ""),
                "pubDate": item.get("pubDate", ""),
                "description": item.get("description", ""),
                "isbn": item.get("isbn", ""),
                "isbn13": item.get("isbn13", ""),
                "itemId": item.get("itemId", ""),
                "priceSales": item.get("priceSales", ""),
                "priceStandard": item.get("priceStandard", ""),
                "categoryId": item.get("categoryId", ""),
                "categoryName": item.get("categoryName", ""),
                "publisher": item.get("publisher", ""),
                "customerReviewRank": item.get("customerReviewRank", ""),
            }
            for item in data["item"]
        ]
        return pd.DataFrame(items)
    else:
        print("API 호출 성공, 그러나 반환된 데이터가 없습니다.")
        return pd.DataFrame()  # 빈 데이터프레임 반환

def collect_books_no_limit(ttbkey, query=""):
    df = pd.DataFrame(columns=[
        "title", "link", "author", "pubDate", "description", "isbn", "isbn13",
        "itemId", "priceSales", "priceStandard", "categoryId", "categoryName",
        "publisher", "customerReviewRank",
    ])

    start = 1
    max_results_per_call = 50

    while True:  # start <= 5 : 변경된 부분: 5회 반복으로 제한
        new_data = search_books(ttbkey, query, start=start, max_results=max_results_per_call)

        if new_data.empty:
            print("데이터 수집이 중단되었습니다. (API 호출 실패 또는 데이터 부족)")
            break

        df = pd.concat([df, new_data]).drop_duplicates(subset=["itemId"], ignore_index=True)
        print(start)
        start += 1

        if len(new_data) < max_results_per_call:
            print("더 이상 데이터를 수집할 수 없습니다.")
            break

    return df

In [9]:
collect_books_no_limit(TTBKEY, "python")

1
더 이상 데이터를 수집할 수 없습니다.


,title,link,author,pubDate,description,isbn,isbn13,itemId,priceSales,priceStandard,categoryId,categoryName,publisher,customerReviewRank
0,"밑바닥부터 시작하는 딥러닝 1 - 파이썬으로 익히는 딥러닝 이론과 구현, 리마스터판",http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"사이토 고키 (지은이), 개앞맵시(이복연) (옮긴이)",2025-01-24,딥러닝의 핵심 개념을 ‘밑바닥부터’ 구현해보며 기초를 한 걸음씩 탄탄하게 다질 수 ...,K222036504,9791169213387,356399264,23400,26000,6794,국내도서>컴퓨터/모바일>컴퓨터 공학>데이터베이스 개론,한빛미디어,9
1,핵심만 쏙쏙! 쉽게 배우는 파이썬,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"양숙희, 장은실, 오경선 (지은이)",2025-01-24,파이썬의 기초를 다지는 데 꼭 필요한 내용을 선별하여 복잡함을 최소화했다. “왜 파...,8989383129,9788989383123,356277845,24000,24000,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,배움터,0
2,AI 시대의 파이썬 데이터 분석 - 다양한 예제로 실습하고 프로젝트로 완성한다!,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,오세종 (지은이),2025-01-19,"파이썬을 활용하여 데이터 분석을 배우려는 독자를 대상으로 하며, 다양한 실습을 통해...",K182036395,9791173400087,356316564,32000,32000,179288,국내도서>수험서/자격증>민간자격>데이터분석,한빛아카데미(교재),0
3,파이썬 마스터 - 실생활 융합 예제로 배우는,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,"김종훈, 김동건 (지은이)",2025-01-17,파이썬 프로그래밍 기초와 데이터 분석을 한 권으로 마스터하는 파이썬 기본서. 파이썬...,K522036896,9791173400018,355902182,32000,32000,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,한빛아카데미(교재),0
4,파이썬 워크북,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,안기수 (지은이),2025-01-17,파이썬 언어를 쉽게 이해하기 위해 워크북으로 실습하는 프로그래밍 입문서다. 코딩을 ...,K712036278,9791192932682,355394344,15000,15000,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,생능,0
5,최신 파이썬 완전정복 : 기초에서 심화까지,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,강환수 (지은이),2025-01-12,,K992035710,9791156002352,353808974,34000,34000,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,홍릉(홍릉과학출판사),0
6,"파이썬 업무 자동화 일잘러 되기 + 챗GPT - 파이썬, 엑셀, 워드, 파워포인트,...",http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,메이허 (지은이),2025-01-10,"반복 작업과 대량의 데이터 처리 작업에 지쳐버린 당신에게 이 책을 선물한다. 파일,...",K802035420,9791194383130,354137053,32400,36000,6357,국내도서>컴퓨터/모바일>프로그래밍 언어>프로그래밍 언어 기타,골든래빗(주),0
7,파이썬으로 제어하는 마이크로비트 - 개정판,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,북마크 (지은이),2025-01-02,,K412036098,9791141971335,356006960,10900,10900,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,부크크(bookk),0
8,마이크로파이썬을 활용한 사물인터넷 - 챗GPT로 코딩하기,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,양재삼 (지은이),2025-01-02,간결한 언어인 마이크로파이썬을 활용해 누구나 쉽게 사물인터넷(IoT)을 시작할 수 ...,K932036178,9791173400049,355341238,28000,28000,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,한빛아카데미(교재),0
9,Python 리터러시,http://www.aladin.co.kr/shop/wproduct.aspx?Ite...,민상준 (지은이),2024-12-30,,K852036099,9791141970338,356015432,19200,19200,6734,국내도서>컴퓨터/모바일>프로그래밍 언어>파이썬,부크크(bookk),0
